# MDPs, Trajetórias e Retornos


In [2]:
import gym
import time
import numpy as np

# vamos focar nesses três ambientes por serem mais simples
#env = gym.make("MountainCar-v0")
#env = gym.make("CartPole-v1")
env = gym.make("Taxi-v3")
#env = gym.make("FrozenLake-v1")

![Figura mostrando interação agente(política)-ambiente](figura_mdp.png "Interação agente-ambiente")

# 1. Episódio e Trajetória

Um *episódio* é uma execução da tarefa (ou do ambiente gym). 

E a *trajetória* é a sequência de estados (observações), ações e recompensas do episódio. Assumindo um episódio de $n$ passos (ações aplicadas):

$S_0 \rightarrow A_0 \rightarrow R_1 \rightarrow S_1 \rightarrow A_1 \rightarrow R_2 \rightarrow S_2 \rightarrow \cdots S_{n-1} \rightarrow A_{n-1} \rightarrow R_n \rightarrow S_n$

Vamos ilustrar um episódio em um MDP usando o ambiente *"env"* escolhido no código acima. 

Estamos assumindo que o episódio encerrou de fato (chegou em um estado final) em *$n$="TOTAL_STEPS"* passos.

In [3]:
TOTAL_STEPS = 5

i = 0
obs = env.reset()
print(f"S0 = {obs}")

done = False

# roda apenas alguns passos
for i in range(0,TOTAL_STEPS):
    #env.render()
    action = env.action_space.sample()
    print(f" A{i} = {action}")

    next_obs, reward, done, info = env.step(action)

    print(f"  R{i+1} = {reward}")
    print(f"S{i+1} = {next_obs}")

    obs = next_obs
    time.sleep(0.1)

env.close()

S0 = 62
 A0 = 4
  R1 = -10
S1 = 62
 A1 = 1
  R2 = -1
S2 = 62
 A2 = 3
  R3 = -1
S3 = 42
 A3 = 3
  R4 = -1
S4 = 42
 A4 = 5
  R5 = -10
S5 = 42


Os detalhes do *episódio* que mostramos acima são chamamos de *trajectory* ou *rollout*.

Dependendo do algoritmo, vamos precisar analisar essas informações em trios (S,A,R) ou quádruplas (S,A,R,S) ou até quíntuplas (S,A,R,S',A').

Abaixo, vamos agrupar e guardar em trio, para preparar para o algoritmo Monte Carlo. E vamos rodar 1 episódio completo.

In [4]:
obs = env.reset()
trajectory = [] 

done = False

while not done:
    #env.render()
    action = env.action_space.sample()

    next_obs, reward, done, info = env.step(action)

    trajectory.append( (obs, action, reward) )

    obs = next_obs
    time.sleep(0.1)

env.close()
#trajectory.append( (obs, None, None) )

print("Trajetória como sequência de trios (STATE, ACTION, REWARD):")
print(trajectory)

Trajetória como sequência de trios (STATE, ACTION, REWARD):
[(288, 4, -10), (288, 0, -1), (388, 0, -1), (488, 0, -1), (488, 1, -1), (388, 4, -10), (388, 5, -10), (388, 5, -10), (388, 3, -1), (368, 5, -10), (368, 2, -1), (388, 0, -1), (488, 4, -10), (488, 2, -1), (488, 3, -1), (468, 0, -1), (468, 0, -1), (468, 4, -10), (468, 5, -10), (468, 5, -10), (468, 5, -10), (468, 0, -1), (468, 1, -1), (368, 1, -1), (268, 4, -10), (268, 3, -1), (248, 0, -1), (348, 1, -1), (248, 2, -1), (268, 0, -1), (368, 5, -10), (368, 0, -1), (468, 2, -1), (488, 1, -1), (388, 2, -1), (388, 0, -1), (488, 5, -10), (488, 1, -1), (388, 4, -10), (388, 2, -1), (388, 2, -1), (388, 2, -1), (388, 5, -10), (388, 1, -1), (288, 1, -1), (188, 0, -1), (288, 1, -1), (188, 1, -1), (88, 1, -1), (88, 0, -1), (188, 4, -10), (188, 5, -10), (188, 5, -10), (188, 1, -1), (88, 5, -10), (88, 3, -1), (68, 3, -1), (48, 3, -1), (48, 0, -1), (148, 2, -1), (168, 3, -1), (148, 2, -1), (168, 4, -10), (168, 3, -1), (148, 3, -1), (148, 4, -10), (

# 2.Calcular os Retornos

O *retorno (final)* $G$ é uma medida da recompensa total obtida ao longo de um episódio. 

Em um MDP, o objetivo é otimizar o valor médio de $G$, para infinitos episódios.

Para isso, vamos assumir a política abaixo, que escolhe a ação $0$ com 50% de probabilidade, ou outra ação, caso contrário.


In [5]:
num_actions = env.action_space.n

def policy(state):
    x = np.random.random()
    if x <= 0.5:
        return 0
    else:
        return np.random.randint(1, num_actions)


E vamos criar uma trajetória com esta política:

In [6]:
obs = env.reset()
trajectory = [] 

done = False
while not done:
    action = policy(obs)
    next_obs, reward, done, info = env.step(action)
    trajectory.append( (obs, action, reward) )
    obs = next_obs

env.close()
#trajectory.append( (obs, None, None) )

print("Trajetória:")
print(trajectory)

Trajetória:
[(106, 0, -1), (206, 0, -1), (306, 4, -10), (306, 0, -1), (406, 0, -1), (406, 5, -10), (406, 5, -10), (406, 2, -1), (406, 0, -1), (406, 0, -1), (406, 0, -1), (406, 4, -10), (406, 0, -1), (406, 0, -1), (406, 0, -1), (406, 0, -1), (406, 0, -1), (406, 5, -10), (406, 0, -1), (406, 0, -1), (406, 0, -1), (406, 0, -1), (406, 0, -1), (406, 0, -1), (406, 1, -1), (306, 3, -1), (306, 3, -1), (306, 0, -1), (406, 0, -1), (406, 0, -1), (406, 0, -1), (406, 0, -1), (406, 0, -1), (406, 1, -1), (306, 0, -1), (406, 3, -1), (406, 0, -1), (406, 3, -1), (406, 4, -10), (406, 1, -1), (306, 0, -1), (406, 0, -1), (406, 3, -1), (406, 5, -10), (406, 5, -10), (406, 0, -1), (406, 4, -10), (406, 2, -1), (406, 4, -10), (406, 4, -10), (406, 2, -1), (406, 0, -1), (406, 0, -1), (406, 3, -1), (406, 1, -1), (306, 2, -1), (306, 0, -1), (406, 4, -10), (406, 5, -10), (406, 0, -1), (406, 2, -1), (406, 0, -1), (406, 2, -1), (406, 0, -1), (406, 3, -1), (406, 4, -10), (406, 0, -1), (406, 2, -1), (406, 5, -10), (406, 

### 2.1 Retorno final do episódio ($G$)



Para um episódio com $n$ passos, o *retorno* (não-descontado) é calculado assim:

$ G = R_1 + R_2 + R_3 + \cdots + R_n = \displaystyle\sum_{i=1}^{n} R_i$

In [7]:
sum_rewards = 0.0
for (s, a, r) in trajectory:
    sum_rewards += r

print("Retorno não-descontado:", sum_rewards)

Retorno não-descontado: -623.0


O mais usado é o *retorno descontado* de um episódio.

Neste caso, $G$ é uma soma que "atenua" recompensas mais distantes, valorizando mais as recompensas iniciais. (Você prefere receber 100 reais agora, de uma vez, ou em 100 parcelas de 1 real?)

Para isso, a cada passo, a recompensa tem uma redução dada por um parâmetro $\gamma\;$, tal que $0 < \gamma \leq 1$.

Para um episódio com $n$ passos, o *retorno descontado* é calculado assim:

$ G = R_1 + \gamma R_2 + \gamma^2 R_3 + \cdots + \gamma^{(n-1)} R_n = \displaystyle\sum_{t=1}^{n} \gamma^{(t-1)} R_t$

In [8]:
GAMMA = 0.95  # você escolhe esse valor

step = 0
discounted_sum_rewards = 0.0
for (s, a, r) in trajectory:
    discounted_sum_rewards += (GAMMA ** step) * reward
    step += 1

print("Retorno descontado:", discounted_sum_rewards)

Retorno descontado: -199.99298946675006


### 2.2 Retornos intermediários a cada passo ($G_i$)



Também podemos calcular um retorno parcial, de um certo episódio, a partir de um passo específico $i$ do episódio:

- $ G_0 = R_1 + \gamma R_2 + \gamma^2 R_3 + \gamma^3 R_4 + \gamma^4 R_5 + \cdots \;\;= G$ 
- $ G_1 = \;\;\;\;\;\;\;\;\;  R_2 + \;\gamma R_3 + \;\gamma^2 R_4 + \;\gamma^3 R_5  + \cdots $ 
- $ G_2 = \;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\; R_3 + \;\gamma R_4 + \;\gamma^2 R_5 + \cdots $ 
- $\cdots$ 
- $ G_n = 0 $

Propriedade:
- $ G_{i-1} = R_i + \gamma G_i $

In [9]:
# calcula os retornos a cada passo (G_i, para cada i=0...n) do episódio completo
G = 0.0
all_Gs = [ G ]
for (s, a, r) in reversed(trajectory):   
    G = r + GAMMA*G
    all_Gs.insert(0, G)

print(all_Gs)

[-59.003513834546595, -61.05633035215431, -63.21718984437296, -56.01809457302417, -57.91378376107808, -59.90924606429272, -52.53604848872918, -44.77478788287282, -46.078724087234555, -47.45128851287848, -48.89609317145103, -50.41694018047477, -42.54414755839449, -43.730681640415256, -44.97966488464764, -46.294384089102785, -47.67829904116083, -49.13505162227456, -41.19479118134164, -42.31030650667542, -43.484533164921494, -44.72056122623315, -46.0216433960349, -47.39120357477358, -48.832845868182716, -50.35036407177128, -51.94775165449609, -53.62921226789062, -55.39917080830592, -57.262285061374655, -59.223457959341744, -61.28785048351763, -63.46089524580803, -65.7483107850611, -68.15611661585379, -70.69064906931979, -73.35857796770505, -76.16692417653164, -79.12307808055962, -72.7611348216417, -75.53803665435969, -78.46109121511546, -81.53799075275312, -84.77683237131907, -78.71245512770429, -72.32890013442557, -75.08305277307954, -68.50847660324163, -71.06155431920172, -64.2753203360

# 3. Funções de Valor

São funções que não fazem parte da essência de um MDP, mas são úteis para criar algoritmos.

Todas elas fazem avaliações dos retornos esperados para uma política específica! Ambas são calculadas a partir dos retornos intermediários.

Veremos dois tipos de função de valor, a seguir. 

### 3.1 Função de valor do estado V(s)

Calcula o retorno esperado a partir de cada estado $s$.

De forma uma pouco mais matemática:

$V(s) = E[G_t | S_t=s]$ 

---

Uma explicação mais algorítmica do valor de $V(s)$ para um $s$ específico, seria esta:
1. rode infinitos episódios com a política
2. para cada episódio:
   - examine se o estado $s$ ocorre em algum passo $t$ (qualquer)
   - se ocorrer, salve os possíveis valores $G_t$
3. Tire a média dos valores salvos

---

Vamos implementar esta ideia, rodando apenas 1 episódio, e vamos calcular o $V(s)$ para todo estado que encontrarmos. Para isso, vamos anexar cada retorno a um dicionário "returns-history" (um histórico dos retornos) indexado pelo $s$ onde se originou cada retorno intermediário.

Esta implementação assume ambiente de *estado discreto*.

In [10]:
returns_history = dict()
V = np.zeros(env.observation_space.n)

# calcula os retornos a cada passo (G_i, para cada i=0...n) do episódio completo
G = 0.0
for (s, a, r) in reversed(trajectory):   
    G = r + GAMMA*G
    
    if s in returns_history.keys():
        returns_history[s].append(G)
    else:
        returns_history[s] = [ G ]
    
    V[s] = np.mean( returns_history[s] )

print(V)

[  0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.         

### 3.2 Função de valor do estado-ação Q(s,a)

De maneira análoga ao $V(s)$, o $Q(s,a)$ representa o retorno esperado a partir do par $(s,a)$. 

Em outras palavras, Q(s,a) responde esta pergunta:

*"Quando estava no estado $s$ e fez a ação $a$, qual o retorno esperado?"*

A definição e a forma de calcular é análoga ao $V(s)$, mas vamos usar um "returns_history" indexado por cada par $(s,a)$ onde se originou cada retorno $G_t$.

In [13]:
returns_history = dict()
Q = np.zeros(shape=(env.observation_space.n, env.action_space.n))

# calcula os retornos a cada passo (G_i, para cada i=0...n) do episódio completo
G = 0.0
for (s, a, r) in reversed(trajectory):   
    G = r + GAMMA*G
    
    if (s,a) in returns_history.keys():
        returns_history[s,a].append(G)
    else:
        returns_history[s,a] = [ G ]
    
    Q[s,a] = np.mean( returns_history[s,a] )

print(Q)

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


# 4. Introdução aos Métodos Baseados em Q-table

Suponha que você partiu de uma política qualquer (provavelmente ruim) e calculou o *Q* dessa política.

De que forma você poderia melhorá-la?

Ou, como escolher a melhor ação a cada estado, usando Q?

---

No próxima parte do curso, veremos um método Monte-Carlo para RL, onde a política é implicitamente representada por Q. 

Este método repete N vezes esses passos:
1. Roda um episódio, usando a política representada pela tabela Q
   - salva a trajetória
1. Depois, usa a trajetória para calcular os valores de $G_i$ e usa esses valores para atualizar Q
   - ao atualizar Q, a política também muda